# 2 - Veri Seti Hazırlama

[cite_start]Bu projede RAG sisteminin temeli olarak, yüklü olan 3 adet kısa motivasyonel metin dosyası (motivasyon1.txt, motivasyon2.txt, motivasyon3.txt) kullanılmıştır[cite: 17]. [cite_start]Hazır veri setleri kullanılmıştır ve bu durumda veri setini anlatmak yeterlidir[cite: 17, 18].

**Metodoloji ve İçerik:**
* Veri setleri **hazırdır** ve elle oluşturulmuştur. [cite_start](Hazır veri setleri kullanmakta hiçbir sakınca yoktur [cite: 17]).
* **İçerik:** Metinler, kendine inanma, küçük adımlar, vazgeçmeme, umut ve duygusal kabul gibi temaları içermektedir.
* Veri setinin basit tutulması, RAG mimarisinin temel çalışma prensiplerini göstermek amacıyla tercih edilmiştir.

# 3 - Çalışma Kılavuzu

Bu proje, Google Colaboratory ortamında çalışmak üzere tasarlanmıştır. Projeyi çalıştırmak için aşağıdaki adımları sırasıyla takip edin:

## Kurulum ve Çalıştırma

1.  **Gerekli Kütüphanelerin Kurulumu:** Colab ortamında, not defterinin başında yer alan hücredeki `!pip install [kullanılan kütüphaneler]` komutunu çalıştırarak gerekli kütüphaneleri (örneğin `langchain`, `chromadb`, `google-genai` vb.) kurun.
2.  **Google Drive Bağlantısı:** Veri setinin (motivasyonel metinler) Google Drive'da saklandığı varsayılmıştır. Not defterindeki ilgili hücreyi çalıştırarak Google Drive'a bağlantı kurun.
3.  **API Anahtarı (Key) Tanımlama:** Gemini API Anahtarınızı (Key), not defterinde belirtilen yere (örneğin çevre değişkeni olarak) tanımlayın.
4.  **Sırasıyla Çalıştırma:** Kurulum ve bağlantılar tamamlandıktan sonra, not defterini yukarıdan aşağıya doğru **sırasıyla** çalıştırın. Tüm hücreler çalıştırıldığında, not defterinin en sonundaki hücrede chatbot arayüzü aktif olacaktır.

## Requirements.txt (Colab Uyarısı)

Proje Colab üzerinde çalıştığı için ayrı bir `requirements.txt` dosyası zorunlu değildir. Gerekli tüm bağımlılıklar not defterinin ilk hücresinde `!pip install` komutları ile belirtilmiştir.

# 4 - Çözüm Mimariniz (RAG Mimarisi)

## Çözülen Problem
Bu proje, büyük bir dil modelinin (LLM) bilgi kesintisi yaşaması (güncel veya spesifik bir alanda bilgi sahibi olmaması) sorununu çözmeyi amaçlar. Motivasyonel metinler üzerine kurulan chatbot, sadece bu metinlere dayanarak, konu dışına çıkmayan, tutarlı ve ilgili yanıtlar üretir.

## Mimarinin Detayları (RAG Akışı)
Projemiz, Retrieval Augmented Generation (RAG) mimarisi üzerine kurulmuştur. Akış aşağıdaki gibi işler:

1.  **Veri Yükleme ve Parçalama (Loading & Splitting):** Yüklenen motivasyonel metinler (kaynak belgeler), LLM'in bağlam penceresine sığacak şekilde küçük parçalara (chunks) ayrılır.
2.  **Vektörleştirme (Embedding):** Her metin parçası, bir **Embedding Modeli** (örneğin Google'a ait bir model) kullanılarak sayısal vektörlere dönüştürülür. Bu vektörler, metnin anlamsal içeriğini temsil eder.
3.  **Vektör Veritabanı Oluşturma:** Oluşturulan bu vektörler, hızlı arama yapmak amacıyla **Vektör Veritabanına** (örneğin ChromaDB/FAISS) kaydedilir.
4.  **Sorgu Vektörleştirme (Query Embedding):** Kullanıcı bir soru sorduğunda, bu soru da aynı Embedding Modeli kullanılarak vektöre dönüştürülür.
5.  **Geri Çağırma (Retrieval):** Kullanıcı sorgusunun vektörü, Vektör Veritabanı'nda en benzer (semantik olarak en yakın) metin parçalarını bulmak için kullanılır.
6.  **Yanıt Üretme (Generation):** Bulunan ilgili metin parçaları (bağlam/context) ve kullanıcının sorusu, birleştirilerek **LLM'e** (örneğin **Gemini API**) gönderilir. LLM, sadece bu bağlamı kullanarak nihai yanıtı üretir.

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [9]:
import os

# Google Drive'daki klasörün yolunu buraya yaz
veri_klasoru = "/content/drive/MyDrive/Motivasyon Verileri"

tum_cumleler = []

for dosya in os.listdir(veri_klasoru):
    dosya_yolu = os.path.join(veri_klasoru, dosya)
    with open(dosya_yolu, "r", encoding="utf-8") as f:
        satirlar = f.readlines()
        tum_cumleler.extend(satirlar)

print("Toplam", len(tum_cumleler), "motivasyon cümlesi yüklendi!")


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Motivasyon Verileri'

In [10]:
!ls /content/drive/MyDrive


''\''Belge (2)'\'' dosyasını sizinle paylaşıyorum'
''\''Belge (2)'\'' dosyasını sizinle paylaşıyorum (1)'
''\''Belge (3)'\'' dosyasını sizinle paylaşıyorum'
'‎Ceyda Kökcü kişisinden ses'
'‎Ceyda Kökcü kişisinden ses (1)'
'‎Ceyda Kökcü kişisinden ses (2)'
'‎Ceyda Kökcü kişisinden ses (3)'
'‎Ceyda Kökcü kişisinden ses (4)'
'Colab Notebooks'
'Google Earth'
 HW5e_PreIntermediate_Test_Unit_3A.docx
 IMG-20240813-WA0026.jpg
 IMG_5099.zip
 inbound6823513440691173807.pdf
'Irmak Ceyda Elif gitar'
'Irmak  ceyda şarkı'
 Irmak_Ceyda_Vocal.wav
'LALELİ GAYRİMENKUL '
 motivasyon_verileri
 SATILIK_20250722_154901_0000.png
 Screenshot_20250506_200420_Word.jpg
'‎Zeliha Nur ile WhatsApp Sohbeti'
 Zeliş


In [11]:
# Google Drive'daki klasörün içeriğini göster
!ls -la "/content/drive/MyDrive/motivasyon_verileri"


total 2
-rw------- 1 root root 175 Oct 11 16:11 motivasyon1.txt
-rw------- 1 root root 124 Oct 11 16:16 motivasyon2.txt
-rw------- 1 root root 166 Oct 11 16:52 motivasyon3.txt


In [12]:
import os

veri_klasoru = "/content/drive/MyDrive/motivasyon_verileri"  # senin klasörün

tum_cumleler = []

for dosya in os.listdir(veri_klasoru):
    dosya_yolu = os.path.join(veri_klasoru, dosya)
    try:
        with open(dosya_yolu, "r", encoding="utf-8") as f:
            # Her satırı temizle ve boşsa atla
            satirlar = [s.strip() for s in f.readlines() if s.strip()]
            tum_cumleler.extend(satirlar)
    except Exception as e:
        print(f"Hata okurken {dosya}: {e}")

print("Yüklü dosyalar:", os.listdir(veri_klasoru))
print("Toplam", len(tum_cumleler), "motivasyon cümlesi yüklendi!")
# İlk 5 cümleyi göster
for i, c in enumerate(tum_cumleler[:5], 1):
    print(i, "-", c)


Yüklü dosyalar: ['motivasyon1.txt', 'motivasyon2.txt', 'motivasyon3.txt']
Toplam 9 motivasyon cümlesi yüklendi!
1 - ﻿Kendine inan, çünkü sen düşündüğünden daha güçlüsün.
2 - Bugün kötü olabilir ama yarın yeni bir başlangıç.
3 - Küçük adımlar büyük farklar yaratır.
4 - ﻿Vazgeçmek kolaydır ama devam etmek kazandırır.
5 - Her başarı, bir hayalle başlar..


In [14]:
import random

def motivasyon_mesaji():
    return random.choice(tum_cumleler)

print("Motivasyon Chatbot'u başlatıldı. Sohbeti sonlandırmak için 'çık' yazın.\n")

while True:
    try:
        mesaj = input("Sen: ")
    except Exception:
        # Colab'de input() bazen farklı davranır; hata alırsan durdurmak için Kernel -> Restart yapabilir veya hücreyi durdurabilirsin.
        print("Girdi alınamadı. Hücreyi durdurup tekrar deneyin.")
        break

    if not mesaj:
        continue
    if mesaj.lower() in ["çık", "bitir", "hoşça kal"]:
        print("Chatbot: Görüşürüz 💛")
        break
    else:
        cevap = motivasyon_mesaji()
        print("Chatbot:", cevap)


Motivasyon Chatbot'u başlatıldı. Sohbeti sonlandırmak için 'çık' yazın.

Sen: 


KeyboardInterrupt: Interrupted by user

In [15]:
import random

def motivasyon_mesaji():
    return random.choice(tum_cumleler)

print("Motivasyon Chatbot'u başlatıldı. Sohbeti sonlandırmak için 'çık' yazın.\n")

# Colab'de çalışması için input yerine while + prompt yöntemi
for _ in range(10):  # 10 kereye kadar konuşma yapabilirsin
    mesaj = input("Sen: ")

    if mesaj.lower() in ["çık", "bitir", "hoşça kal"]:
        print("Chatbot: Görüşürüz 💛")
        break
    else:
        cevap = motivasyon_mesaji()
        print("Chatbot:", cevap)


Motivasyon Chatbot'u başlatıldı. Sohbeti sonlandırmak için 'çık' yazın.

Sen: çık
Chatbot: Görüşürüz 💛
